In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

In [6]:
service = webdriver.ChromeService(executable_path = 'chromedriver')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome()

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
import logging
import time
from datetime import datetime, timedelta
import pytz

In [8]:
def day_to_weekday_num(day_name):
    """Converts a day name to its corresponding weekday number."""
    days = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
    return days.index(day_name.lower())

def get_target_date_xpath(target_day=None, timezone_str="America/Chicago"):
    local_timezone = pytz.timezone(timezone_str)
    today = datetime.now(local_timezone)

    if target_day:
        # Convert the desired day to its corresponding weekday number
        target_day_num = day_to_weekday_num(target_day)
        days_ahead = (target_day_num - today.weekday() + 7) % 7
        if days_ahead == 0:
            days_ahead = 7  # If today is the desired day, move a week ahead
    else:
        days_ahead = 7  # Default behavior

    target_date = today + timedelta(days=days_ahead)
    day = target_date.day
    month = target_date.month - 1  # Adjust month for zero-based index in HTML

    date_xpath = f"//td[@data-month='{month}'][a[text()='{day}']]"
    return date_xpath

def convert_to_24_hour_format(time_str):
    # Converts a time string in the format of 'hh:mm AM/PM' to 24-hour format.

    return datetime.strptime(time_str, '%I:%M %p').strftime('%H:%M')


def get_multiple_courses():
    if datetime.today().weekday() == 6:  # Sunday is represented by 6
        return 5
    else:
        return 1


In [9]:
# Define your preferences here
PREFERENCES = {
    'primary_course': 'North',
    'secondary_course': 'Northback',
    'tertiary_course': 'Original Front to North Front',
    'preferred_start_time': '07:30',
    'preferred_end_time': '08:29'
}

def select_timeslot(driver):
    start_time = PREFERENCES['preferred_start_time']
    end_time = PREFERENCES['preferred_end_time']

    while True:
        # Check primary preference
        slot = find_slot_on_course(driver, PREFERENCES['primary_course'], start_time, end_time)
        if slot:
            return slot

        # Check secondary preference
        slot = find_slot_on_course(driver, PREFERENCES['secondary_course'], start_time, end_time)
        if slot:
            return slot

        # Check tertiary preference
        slot = find_slot_on_course(driver, PREFERENCES['tertiary_course'], start_time, end_time)
        if slot:
            return slot

        # If none of the preferred courses are found, choose the first available slot
        slot = find_first_available_slot(driver, start_time, end_time)
        if slot:
            return slot

        # No suitable slot found within this hour. Increment the time range by one hour.
        start_time = increment_hour(start_time)
        end_time = increment_hour(end_time)

        # Stopping condition: Stop after checking 12:30 to 1:29 (or customize as required)
        if start_time == '13:30':
            break

    return None


def increment_hour(time_str):
    """Given a time string in HH:MM format, increment the hour by 1."""
    hour, minute = map(int, time_str.split(":"))
    hour += 1
    return f"{hour:02}:{minute:02}"

def find_slot_on_course(driver, start_time, end_time):
    """
    This function finds the first available slot within the given time range.
    """
    # Your logic here, which chooses the first available time slot within the time range
    # on the designated course
    pass

def find_first_available_slot(driver, start_time, end_time):
    """
    This function finds the first available slot within the given time range, without considering the course.
    """
    # Your logic here, which might be similar to the find_slot_on_course function
    # but without filtering by the course name.
    pass

# ... rest of the functions ...


In [10]:
# This function contains your logic for locating the time slots.
# It's separate so you can call it for both preferred and secondary courses.
def search_for_time_slot(course_name):
    global start_clicking
    global time_text

    try:
        rows = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located(
                (By.XPATH, f"//div[contains(@class, 'rwdTr') and div[@class='sN rwdTd' and text()='{course_name}']]"))
        )
    except TimeoutException:
        print(f"No rows found for course: {course_name}")
        return False

    for row in rows:
        try:
            # Extract the time slot from the <a> tag or the <div> tag
            try:
                time_element = row.find_element(By.XPATH, ".//div[@class='sT rwdTd']/a")
            except NoSuchElementException:
                time_element = row.find_element(By.XPATH, ".//div[@class='sT rwdTd']/div[@class='time_slot']")

            time_text = convert_to_24_hour_format(time_element.text.strip())

            if time_text < current_user.preferred_timeslot:
                continue

            if multiple_courses > 1:
                select_element = row.find_element(By.XPATH, ".//div[@class='sS rwdTd']/select")
                Select(select_element).select_by_value(str(multiple_courses))
                WebDriverWait(driver, 10).until(EC.staleness_of(time_element))
                return True  # Exit the loop as you've selected the required option and the page might have navigated
            elif isinstance(time_element,
                            webdriver.remote.webelement.WebElement) and "teetime_button" in time_element.get_attribute(
                "class"):
                time_element.click()
                return True  # Slot was found and clicked

        except NoSuchElementException as e:
            print(e)
            continue
        except StaleElementReferenceException:
            # Handle the stale element exception, you can log it or wait and then continue
            time.sleep(2)  # Wait for a short period before retrying
            continue

    return False

In [11]:
class User:
    def __init__(self, user_name: str, user_password: str, user_alt_attribute: str, preferred_timeslot: str,
                 preferred_courses: list):
        self.user_name = user_name
        self.user_password = user_password  # Dont keep it this way
        self.user_alt_attribute = user_alt_attribute
        self.preferred_timeslot = preferred_timeslot
        self.preferred_courses = preferred_courses

In [12]:
# Define ParlowS User class as a test user
current_user = User(user_name='PartlowS',
                    user_password='xfu*fyb6RBC_cyx8mcg',
                    user_alt_attribute='Scott Partlow',
                    preferred_timeslot='06:00',
                    preferred_courses=['North', 'Original Front to North Front',
                                       'Northback'])  # In Order - first gets priority !important

In [13]:
multiple_courses = 1

# -Optional- select a day to run the script on by typing a day (string) as a parameter for this function
date_xpath = get_target_date_xpath()

# Set up the driver
# chrome_options = Options()
# chrome_options.add_argument("--headless")
# chrome_options.add_argument('--window-size=1920x1080')
# chrome_options.add_argument("--no-sandbox")
# chrome_options.add_argument("--disable-dev-shm-usage")
# driver = webdriver.Chrome(options=chrome_options)
driver = webdriver.Chrome()

# Navigate to the login page
driver.get('https://www.onioncreekclub.com/user/login')

In [14]:
# Find the email and password fields, enter the credentials
name_elem = driver.find_element(By.ID, 'edit-name--2')
password_elem = driver.find_element(By.ID, 'edit-pass--2')

name_elem.send_keys(current_user.user_name)
password_elem.send_keys(current_user.user_password)
password_elem.send_keys(Keys.RETURN)

In [15]:
wait = WebDriverWait(driver, 120)
original_window = driver.current_window_handle

# Debug: Log the original window title
print(f"[DEBUG] Original window title: {driver.title}")

[DEBUG] Original window title: PartlowS | Onion Creek Club


In [16]:
# Find the button by its text and href pattern and click it
button_to_click = wait.until(
    EC.presence_of_element_located((By.XPATH,
                                    "//a[contains(@href, '/user/')][contains(@href, '/foretees/login')][contains("
                                    "text(), 'Book a Tee Time Now')]"))
)
button_to_click.click()

In [17]:
wait.until(EC.number_of_windows_to_be(2))

# Get
for window_handle in driver.window_handles:
    if window_handle != original_window:
        driver.switch_to.window(window_handle)

# Debug: Log the current window title after the switch
print(f"[DEBUG] After switch window title: {driver.title}")

[DEBUG] After switch window title: Member Identification


In [ ]:
# Switch to the new tab
wait.until(EC.title_is("Member Identification"))

# Wait for the button with the specified alt attribute and click it
button_to_click = wait.until(
    EC.presence_of_element_located(
        (By.XPATH, f"//a[@alt='{current_user.user_alt_attribute}'][text()='{current_user.user_alt_attribute}']"))
)
button_to_click.click()

In [18]:
wait.until(EC.title_is("Welcome to ForeTees"))

# Navigate to the 'Member_select' page
driver.get('https://www1.foretees.com/v5/onioncreekclub_golf_m56/Member_select')

# Find the date on the calendar and click it
print(date_xpath)
day_to_click = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, date_xpath))
)
day_to_click.click()

slot_found = False
start_clicking = False

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=117.0.5938.92)
Stacktrace:
0   chromedriver                        0x0000000103529ed8 chromedriver + 5091032
1   chromedriver                        0x0000000103520c43 chromedriver + 5053507
2   chromedriver                        0x00000001030bc527 chromedriver + 447783
3   chromedriver                        0x00000001030902f8 chromedriver + 267000
4   chromedriver                        0x00000001031380cf chromedriver + 954575
5   chromedriver                        0x000000010314dcc6 chromedriver + 1043654
6   chromedriver                        0x0000000103132433 chromedriver + 930867
7   chromedriver                        0x00000001030fd042 chromedriver + 712770
8   chromedriver                        0x00000001030fe26e chromedriver + 717422
9   chromedriver                        0x00000001034eb439 chromedriver + 4834361
10  chromedriver   

In [ ]:
selected_course = None

for course in current_user.preferred_courses:
    if search_for_time_slot(course):
        selected_course = course
        print(f'Selected course: {selected_course}')
        # break
    else:  # This block executes if the loop completes without a break
        print(
            f"No available slots found in either the '{current_user.preferred_courses[0]}' or '{current_user.preferred_courses[1]}' course rows.")
        logging.warning(
            f"No available slots found in either the '{current_user.preferred_courses[0]}' or '{current_user.preferred_courses[1]}' course rows.")
        input("Press Return to exit...")
        exit(0)

In [ ]:
if multiple_courses > 1:
    continue_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//button[span[text()='Continue']]"))
    )
    continue_button.click()

In [ ]:
button_to_click = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//a[@class='ftS-playerPrompt standard_button']"))
)

In [ ]:
button_to_click = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//div[@data-fttab='.ftMs-guestTbd']"))
)

In [ ]:
# Step 1: Locate the button you want to click three times
button_to_click = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'ftMs-listItem')][span[text()='X']]"))
)

print(f'X button found. Attempting to click {((4 * multiple_courses) - 1)} times...')

In [ ]:
# Step 2: Click it three times
for _ in range((4 * multiple_courses) - 1):
    driver.execute_script("arguments[0].click();", button_to_click)

In [ ]:
# Step 3: Check for the existence of at least three <div class="playerType">X</div>
player_types = driver.find_elements(By.XPATH, "//div[@class='playerType' and text()='X']")

if len(player_types) >= ((4 * multiple_courses) - 1):
    # Step 4: If the check passes, locate the submit button and click it
    try:
        submit_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.XPATH, "//a[@class='submit_request_button' and text()='Submit Request']"))
        )
        print(
            f"Successfully Booked the {time_text} time slot at the {selected_course} course for {current_user.user_alt_attribute}")
        logging.warning(
            f"Successfully Booked the {time_text} time slot at the {selected_course} course for {current_user.user_alt_attribute}")
        input("Ready to click Submit button")
        submit_button.click()
        exit(0)
    except TimeoutException:
        print(f"Could not locate Submit Button for the {time_text} time slot. Exiting...")
        logging.warning(f"Could not locate Submit Button for the {time_text} time slot.")
        exit(0)
    except Exception as e:
        print(e)